Word2Vec

   - 분류 등과 같이 별도의 레이블이 없이 텍스트 자체만 있어도 학습이 가능
   - Word2Vec의 방식은 주변 관계를 이용하는 것으로 2가지의 방식이 있음
   - 1. CBOW(continuous Bag-of-words) : 주변 단어의 임베딩을 더해서 대상단어를 예측
   - 2. Skip-Gram : 대상 단어의 임베딩으로 주변단어르 예측
      - 일반적으로 CBOW보다 성능이 좋은 편이지만 한번에 여러 단어를 예측해야하기 때문에 비효율적
      - 최근에는 negative sampling이라는 방법을 사용

T-SNE(t-Stochastic Neighbor Embedding)
   - T-SNE은 고차원의 벡터들의 구조를 보존하며 저차원으로 사상하는 차원 축소 알고리즘
   - 단어 임베딩에서도 생성된 고차원 벡터들을 시각화하기 위해 T-SNE 알고이즘을 많이 이용
   - t-sne는 가장 먼저 원 공간의 데이터 유사도와 임베딩 공간의 데이터 유사도를 정의

In [1]:
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True, random_state=1, 
                             remove=('headers', 'footers', 'quotes'))   # word2vec하는거라 필요없는 것들을 제거

documents = dataset.data

print(len(documents))

11314


In [2]:
import re
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

def clean_text(d):
    pattern = r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', d)
    return d

# 불용어 제거
def clean_stopword(d):
    stop_words = stopwords.words('english')
    return ' '.join([w.lower() for w in d.split() if w not in stop_words and len(w) > 3])

def tokenize(d):
    return word_tokenize(d)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bitcamp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bitcamp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Pandas Frame으로 변환
import pandas as pd
news_df = pd.DataFrame({'article':documents})

news_df.replace("", float("NaN"), inplace=True)
news_df.dropna(inplace=True)
print(len(news_df))

11096


In [4]:
# 특수문자 제거
news_df['article'] = news_df['article'].apply(clean_text)

# 불용어 제거
news_df['article'] = news_df['article'].apply(clean_stopword)

# 토크나이저
tokenized_news = news_df['article'].apply(tokenize)
tokenized_news = tokenized_news.to_list()

In [5]:
import numpy as np

drop_news = [index for index, sentence in enumerate(tokenized_news) if len(sentence) <= 1]
news_texts = np.delete(tokenized_news, drop_news, axis=0)
print(len(news_texts))

10991


C:\ProgramData\Anaconda3\envs\py37\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


Gensim을 이용한 Word2Vec

1. CBOW

In [6]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=news_texts, window=3, vector_size=100, min_count=5, workers=4, sg=0)

# window : 앞뒤로 몇개의 단어를 보고 유추할 것인가? 보통 3,4
# vector_size : 
# workers : 병렬로 처리
# sg : CBOW는 0, skip-gram은 0

model.wv.similarity('man', 'woman')

0.8143273

In [7]:
print(model.wv.most_similar(positive=['soldiers']))
print(model.wv.most_similar(positive=['man', 'soldiers'], negative=['woman']))

[('genocide', 0.9754312634468079), ('turks', 0.9705201387405396), ('villages', 0.9637882113456726), ('armenia', 0.9602059721946716), ('kurds', 0.9580553770065308), ('turkish', 0.9491828680038452), ('killed', 0.9473462104797363), ('murdered', 0.9434210658073425), ('civilians', 0.9411264657974243), ('armenian', 0.9391229152679443)]
[('turkey', 0.8646937012672424), ('jew', 0.8494433760643005), ('sadikov', 0.845048189163208), ('thou', 0.8372932076454163), ('destruction', 0.8336199522018433), ('jews', 0.8334668874740601), ('ismailov', 0.8316466212272644), ('lexicon', 0.8303812742233276), ('armenians', 0.8275583982467651), ('nazi', 0.8263252377510071)]


2. Skip-gram

In [8]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=news_texts, window=3, vector_size=100, min_count=5, workers=4, sg=1)
# sg=1로만 변경하면 됨

model.wv.similarity('man', 'woman')

0.8511009

In [9]:
print(model.wv.most_similar(positive=['soldiers']))
print(model.wv.most_similar(positive=['man', 'soldiers'], negative=['woman']))

[('villages', 0.9159388542175293), ('wounded', 0.9120823740959167), ('civilians', 0.9077595472335815), ('massacre', 0.9016125202178955), ('troops', 0.8977922797203064), ('turks', 0.8956711292266846), ('azerbaijanis', 0.8952058553695679), ('raped', 0.8933436870574951), ('kurds', 0.8900163769721985), ('village', 0.889756977558136)]
[('fighters', 0.8396631479263306), ('israelis', 0.8285832405090332), ('stalin', 0.8265437483787537), ('grandparents', 0.8197566866874695), ('saints', 0.8162000179290771), ('arafat', 0.8146632313728333), ('wwii', 0.8143231272697449), ('jew', 0.8141732811927795), ('arabs', 0.8132178783416748), ('proofs', 0.8116437792778015)]


임베딩 벡터 시각화

In [10]:
from gensim.models import KeyedVectors

model.wv.save_word2vec_format('news_w2v')

# 벡터화된 값을 tsv 파일로 저장하기
!python -m gensim.scripts.word2vec2tensor --input news_w2v --output naver_w2v

Embedding Projector : https://projector.tensorflow.org/